In [1]:
!pip install torch 


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install transformers datasets


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install peft


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install optuna tensorboard


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler, DataCollatorWithPadding

import pandas as pd # For loading your CSV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import os
import optuna
import logging
from peft import get_peft_model, LoraConfig, TaskType

# Configure logging and TensorBoard
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
writer = SummaryWriter()

# --- GLOBAL CONSTANTS ---
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"
CHECKPOINT_DIR = "model_checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Define hyperparameters (Optuna will overwrite these later)
BATCH_SIZE = 16
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3

# CRITICAL FIX: Custom Labels for 3-Class Problem
NUM_LABELS = 3
ID2LABEL = {0: "Negative", 1: "Neutral", 2: "Positive"}
LABEL2ID = {"Negative": 0, "Neutral": 1, "Positive": 2}

# From your original code's global constants section
DATA_FILE_PATH = 'cleaned.csv'

In [38]:
# Directory for saving checkpoints
CHECKPOINT_DIR = "model_checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [39]:
# Function for hyperparameter tuning using Optuna
def objective(trial):
    # 1. Hyperparameter suggestions
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    batch_size = trial.suggest_int("batch_size", 8, 32, step=8)

    # Note: Model loading must be updated globally (num_labels=3, ignore_mismatched_sizes=True)
    tokenizer, model = load_model_and_tokenizer(MODEL_NAME)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Initialize the data collator to handle padding at the batch level
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # ------------------------------------------------------------------------
    # CRITICAL FIX: 3-WAY DATA SPLITTING AND CUSTOM DATA LOADING
    # ------------------------------------------------------------------------
    df_full = pd.read_csv(DATA_FILE_PATH)
    # Data Cleaning and Mapping
    columns_to_drop = ['Persona', 'Specific Clause Referenced', 'Unnamed: 4', 'Unnamed: 5']
    df_clean = df_full.drop(columns=columns_to_drop, errors='ignore')
    df_clean.rename(columns={'comment': 'text'}, inplace=True)
    df_clean['label'] = df_clean['label'].map(LABEL2ID)

    # A. Split into Main (80%) and Test (20%)
    df_main, _ = train_test_split(df_clean, test_size=0.2, random_state=42, stratify=df_clean['label']) 
    
    # B. Split Main (80%) into Train (70% total) and Validation (10% total)
    df_train, df_val = train_test_split(df_main, test_size=1/8, random_state=42, stratify=df_main['label'])
    
    train_dataset = Dataset.from_pandas(df_train).remove_columns(['__index_level_0__'])
    val_dataset = Dataset.from_pandas(df_val).remove_columns(['__index_level_0__'])
    
    # ------------------------------------------------------------------------

    # 3. Training Data Loaders
    train_shuffled = train_dataset.shuffle(seed=42) 
    train_dataset_tokenized = preprocess_data(tokenizer, train_shuffled) 
    train_loader = DataLoader(
        train_dataset_tokenized, 
        batch_size=batch_size, 
        shuffle=True, 
        collate_fn=data_collator # CRITICAL FIX: Use DataCollator
    )

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = NUM_EPOCHS * len(train_loader)
    lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    # 5. Train the model (pass the current trial number for status update)
    train_model(model, train_loader, optimizer, lr_scheduler, device, trial.number)

    # 6. Evaluate on the DEDICATED VALIDATION SET
    val_shuffled = val_dataset.shuffle(seed=42)
    val_dataset_tokenized = preprocess_data(tokenizer, val_shuffled) 
    val_loader = DataLoader(
        val_dataset_tokenized, 
        batch_size=batch_size, 
        collate_fn=data_collator # CRITICAL FIX: Use DataCollator
    )

    predictions, labels = evaluate_model(model, val_loader, device)
    
    # CRITICAL FIX: Use 'weighted' F1 score (multi-class)
    validation_f1 = f1_score(labels, predictions, average='weighted', zero_division=0)
    
    # Explicit status output after trial completes
    logging.info("----------------------------------------------------------")
    logging.info(f"TRIAL {trial.number} SUMMARY: SUCCESS")
    logging.info(f"  Learning Rate: {trial.params['learning_rate']:.7f}")
    logging.info(f"  Batch Size: {trial.params['batch_size']}")
    logging.info(f"  Validation F1-Score: {validation_f1:.4f}")
    logging.info("----------------------------------------------------------")
    
    return validation_f1

In [40]:
# Assuming these global constants are defined:
# NUM_LABELS = 3
# ID2LABEL = {0: "Negative", 1: "Neutral", 2: "Positive"}
# LABEL2ID = {"Negative": 0, "Neutral": 1, "Positive": 2}

def load_model_and_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # CRITICAL FIX: Add multi-class configuration and size mismatch override
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=NUM_LABELS,   # Tells the model to use 3 output neurons
        id2label=ID2LABEL,       # Maps prediction IDs (0, 1, 2) to labels
        label2id=LABEL2ID,        # Maps labels to IDs
        ignore_mismatched_sizes=True # Fixes the conflict between old 2-class head and new 3-class head
    )

    # LoRA configuration remains the same
    target_modules = ["q_lin", "k_lin", "v_lin", "out_lin"]
    
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules=target_modules
    )
    model = get_peft_model(model, peft_config)
    logging.info("LoRA model initialized.")

    return tokenizer, model

In [41]:
# Preprocess data
def preprocess_data(tokenizer, dataset):
    def tokenize(batch):
        # CRITICAL FIX: Removed padding=True. 
        # Padding is now handled by the DataCollatorWithPadding in the DataLoader.
        return tokenizer(batch['text'], truncation=True, max_length=128)

    dataset = dataset.map(tokenize, batched=True)
    # The format setting is correct for the Dataset object
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return dataset

In [42]:
def train_model(model, train_loader, optimizer, lr_scheduler, device, trial_identifier):
    model.train()
    
    total_epoch_loss = 0.0
    
    for epoch in range(NUM_EPOCHS):
        logging.info(f"Starting epoch {epoch + 1}/{NUM_EPOCHS}")
        
        # Reset tracking variables for each epoch
        total_epoch_loss = 0.0
        num_batches = 0
        
        for i, batch in enumerate(train_loader):
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                # CRITICAL FIX 1: Use 'labels' key (plural) from DataCollator
                labels = batch['labels'].to(device) 

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                
                # Accumulate loss and batch count for robust averaging
                total_epoch_loss += loss.item()
                num_batches += 1

                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                lr_scheduler.step()

                # CRITICAL FIX 3: Add live status update for continuous feedback
                if i % 50 == 0: 
                    print(f"-> Trial {trial_identifier}, Epoch {epoch+1}/{NUM_EPOCHS}, Batch {i}/{len(train_loader)}. Current Loss: {loss.item():.4f}", end='\r')
                
                writer.add_scalar("Loss/train", loss.item(), epoch * len(train_loader) + i)

            except Exception as e:
                logging.error(f"Error during training: {e}")

        # Save checkpoint
        checkpoint_path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{epoch + 1}.pt")
        torch.save(model.state_dict(), checkpoint_path)
        # Added \n to move the cursor past the live status update before logging
        logging.info(f"\nCheckpoint saved at {checkpoint_path}")

        # CRITICAL FIX 2: Calculate and log the average loss for the epoch
        if num_batches > 0:
            avg_loss = total_epoch_loss / num_batches
            logging.info(f"Epoch {epoch + 1} completed. Average Loss: {avg_loss:.4f}")
        else:
            logging.info(f"Epoch {epoch + 1} completed. No batches processed.")

In [43]:
# Evaluate model
def evaluate_model(model, data_loader, device):
    model.eval()
    predictions, labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                # CRITICAL FIX: Change 'label' to 'labels' to match DataCollator output
                labels.extend(batch['labels'].tolist())

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                predictions.extend(torch.argmax(logits, dim=-1).tolist())

            except Exception as e:
                logging.error(f"Error during evaluation: {e}") 

    return predictions, labels

In [44]:
# Classify text
def classify_text(model, tokenizer, text, device):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    model.eval()
    with torch.no_grad():
        try:
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=-1).item()
            probabilities = torch.softmax(logits, dim=-1).squeeze().tolist()
        except Exception as e:
            logging.error(f"Error during text classification: {e}")
            return None, None

    return predicted_class, probabilities


In [47]:
# Assuming all imports and global constants (NUM_LABELS, ID2LABEL, DATA_FILE_PATH, etc.)
# are defined at the top of your script.

if __name__ == "__main__":
    tokenizer, model = load_model_and_tokenizer(MODEL_NAME)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # ------------------------------------------------------------------------
    # CRITICAL FIX: FINAL 3-WAY DATA SPLITTING FOR TRAINING/TESTING
    # ------------------------------------------------------------------------
    logging.info("Preparing final Train/Test data sets...")
    df_full = pd.read_csv(DATA_FILE_PATH)
    
    # Data Cleaning and Mapping (Must match objective function logic)
    columns_to_drop = ['Persona', 'Specific Clause Referenced', 'Unnamed: 4', 'Unnamed: 5']
    df_clean = df_full.drop(columns=columns_to_drop, errors='ignore')
    df_clean.rename(columns={'comment': 'text'}, inplace=True)
    df_clean['label'] = df_clean['label'].map(LABEL2ID)

    # A. Split into Main (80%) and Test (20%) - FINAL HELD-OUT TEST SET
    df_main, df_test = train_test_split(df_clean, test_size=0.2, random_state=42, stratify=df_clean['label']) 
    
    # B. Split Main (80%) into Train (70% total) and Validation (10% total) - FINAL TRAINING DATA
    df_train, df_val = train_test_split(df_main, test_size=1/8, random_state=42, stratify=df_main['label'])
    
    # Convert to Hugging Face Dataset format
    train_dataset = Dataset.from_pandas(df_train).remove_columns(['__index_level_0__'])
    test_dataset = Dataset.from_pandas(df_test).remove_columns(['__index_level_0__'])
    
    # Initialize the Data Collator (CRITICAL for fixing batching errors)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    # ------------------------------------------------------------------------

    # Hyperparameter tuning
    study = optuna.create_study(direction="maximize")
    # The 'objective' function must contain the full 3-way split logic and evaluate on the VALIDATION set
    study.optimize(objective, n_trials=10) 

    logging.info("Best hyperparameters:")
    logging.info(study.best_params)

    # Load best hyperparameters
    best_learning_rate = study.best_params["learning_rate"]
    best_batch_size = study.best_params["batch_size"]

    # --- FINAL TRAINING (Uses TRAIN SET + BEST HPs) ---
    train_shuffled = train_dataset.shuffle(seed=42)
    train_dataset = preprocess_data(tokenizer, train_shuffled)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=best_batch_size, 
        shuffle=True, 
        collate_fn=data_collator # CRITICAL FIX
    )

    # --- FINAL TEST (Uses TEST SET for UNBIASED EVALUATION) ---
    test_shuffled = test_dataset.shuffle(seed=42)
    test_dataset = preprocess_data(tokenizer, test_shuffled)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=best_batch_size, 
        collate_fn=data_collator # CRITICAL FIX
    )

    optimizer = AdamW(model.parameters(), lr=best_learning_rate)
    num_training_steps = NUM_EPOCHS * len(train_loader)
    lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    logging.info("Starting final fine-tuning of the model.")
    # Pass 'Final' as the trial identifier for the status update
    train_model(model, train_loader, optimizer, lr_scheduler, device, 'Final') 

    logging.info("Evaluating the model on the held-out TEST set.")
    predictions, labels = evaluate_model(model, test_loader, device)

    # CRITICAL FIX 1: Use 'weighted' average for multi-class metrics
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(labels, predictions, average='weighted', zero_division=0)

    logging.info(f"Accuracy: {accuracy * 100:.2f}%")
    # Logging the Weighted F1 Score is better for reporting
    logging.info(f"F1 Score (Weighted): {f1 * 100:.2f}%")
    
    # CRITICAL FIX 2: Print the detailed Classification Report
    logging.info("\nClassification Report (Negative, Neutral, Positive):")
    logging.info(classification_report(labels, predictions, target_names=list(ID2LABEL.values()), zero_division=0))
    
    logging.info("\nConfusion Matrix:")
    logging.info(confusion_matrix(labels, predictions))

    # CRITICAL FIX 3: Correct 3-class sentiment mapping for interactive loop
    while True:
        text = input("\nEnter text for sentiment analysis (or type 'exit' to quit): ")
        if text.lower() == "exit":
            break

        predicted_class, probabilities = classify_text(model, tokenizer, text, device)
        # Use the global ID2LABEL dictionary to correctly map 0, 1, and 2
        sentiment = ID2LABEL.get(predicted_class, "Unknown")

        print(f"Sentiment: {sentiment}")
        print(f"Probabilities: {probabilities}")

    writer.close()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-11-08 00:26:11,897 - INFO - LoRA model initialized.
2025-11-08 00:26:11,908 - INFO - Preparing final Train/Test data sets...
[I 2025-11-08 00:26:11,985] A new study created in memory with name: no-name-12a67651-16ff-466d-a427-6c5f3a01ad64
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did no

-> Trial 0, Epoch 1/3, Batch 700/731. Current Loss: 0.8217

2025-11-08 00:33:58,460 - INFO - 
Checkpoint saved at model_checkpoints\model_epoch_1.pt
2025-11-08 00:33:58,461 - INFO - Epoch 1 completed. Average Loss: 0.8098
2025-11-08 00:33:58,462 - INFO - Starting epoch 2/3


-> Trial 0, Epoch 2/3, Batch 700/731. Current Loss: 0.5758

2025-11-08 00:41:54,938 - INFO - 
Checkpoint saved at model_checkpoints\model_epoch_2.pt
2025-11-08 00:41:54,939 - INFO - Epoch 2 completed. Average Loss: 0.6898
2025-11-08 00:41:54,940 - INFO - Starting epoch 3/3


...


Enter text for sentiment analysis (or type 'exit' to quit):  Introducing a simplified procedure for micro and small enterprises promotes ease of doing business.


Sentiment: Positive
Probabilities: [0.006567909382283688, 0.10578414052724838, 0.8876479864120483]



Enter text for sentiment analysis (or type 'exit' to quit):  The amendment necessitates a revision of the company's existing Memorandum and Articles of Association.


Sentiment: Neutral
Probabilities: [0.02140389010310173, 0.6692243218421936, 0.3093717694282532]



Enter text for sentiment analysis (or type 'exit' to quit):  exit


In [2]:
import torch
import os
import logging
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

# Configure logging (optional)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- CONFIGURATION (MUST MATCH TRAINING SETTINGS) ---
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"
CHECKPOINT_DIR = "model_checkpoints"

# CRITICAL: Path to your saved weights (adjust the epoch number if needed)
LORA_WEIGHTS_PATH = os.path.join(CHECKPOINT_DIR, "model_epoch_3.pt")

NUM_LABELS = 3
ID2LABEL = {0: "Negative", 1: "Neutral", 2: "Positive"}
# ----------------------------------------------------

# --- 1. CLASSIFICATION FUNCTION (COPIED FROM YOUR CODE) ---
def classify_text(model, tokenizer, text, device):
    """Tokenizes input text and returns the predicted class and probabilities."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    model.eval()
    
    with torch.no_grad():
        try:
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=-1).item()
            probabilities = torch.softmax(logits, dim=-1).squeeze().tolist()
            
            # Use the corrected 3-class mapping
            sentiment = ID2LABEL.get(predicted_class, "Unknown")
            
        except Exception as e:
            logging.error(f"Error during text classification: {e}")
            return None, None, None

    return sentiment, predicted_class, probabilities

# --- 2. MODEL LOADING AND INFERENCE SETUP ---
def load_inference_model():
    """Loads the base model, applies LoRA configuration, and loads saved weights."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1. Load Base Model Architecture (with 3-class head)
    logging.info("Loading base model architecture...")
    base_model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        id2label=ID2LABEL,
        ignore_mismatched_sizes=True 
    )

    # 2. Define LoRA Configuration (Must match training config)
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules=["q_lin", "k_lin", "v_lin", "out_lin"]
    )
    
    # 3. Apply the LoRA wrapper to the base model
    model = get_peft_model(base_model, peft_config)

    # 4. Load Saved Weights (Inference)
    logging.info(f"Loading saved LoRA weights from: {LORA_WEIGHTS_PATH}")
    if not os.path.exists(LORA_WEIGHTS_PATH):
        raise FileNotFoundError(f"Checkpoint not found at {LORA_WEIGHTS_PATH}. Did you run the training script successfully?")
        
    # CRITICAL: Load the saved LoRA weights onto the model
    model.load_state_dict(torch.load(LORA_WEIGHTS_PATH, map_location=device))
    
    # 5. Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    model.eval() # Set model to evaluation mode
    model.to(device)
    logging.info("Model loaded and ready for inference.")
    
    return model, tokenizer, device

ModuleNotFoundError: No module named 'torch'

In [1]:
# --- 3. MAIN INTERACTIVE INFERENCE BLOCK ---
if __name__ == "__main__":
    try:
        # Load the trained model only once
        model, tokenizer, device = load_inference_model()
        
        print("\n" + "="*50)
        print("LoRA Sentiment Analyzer Ready. (Type 'exit' to quit)")
        print("="*50 + "\n")

        while True:
            text = input(">> Enter comment: ")
            if text.lower() == "exit":
                break

            sentiment, predicted_class_id, probabilities = classify_text(model, tokenizer, text, device)
            
            if sentiment:
                print(f"   [Prediction]: {sentiment}")
                print(f"   [Probabilities]: Negative: {probabilities[0]:.4f}, Neutral: {probabilities[1]:.4f}, Positive: {probabilities[2]:.4f}")
            else:
                print("   [Error]: Could not process comment.")

    except FileNotFoundError as e:
        print(f"\nFATAL ERROR: {e}")
        print("Please ensure your training script has successfully run and saved 'model_epoch_3.pt' in the 'model_checkpoints' folder.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


An unexpected error occurred: name 'load_inference_model' is not defined
